# The method behind econpizza with heterogenous agents

The methodology extends Auclert et al. (2022, ECMA) to solve the system fully nonlinearly. 
Most heterogeneous agent models can be expressed in terms of the three functions $V$, $D$ and $F$:

$$
\begin{align}
v_t =& V(x_{t-1}, x_t, x_{t+1}, v_{t+1})\\
d_t =& D(x_{t-1}, x_t, x_{t+1}, v_t, d_{t-1})\\
0 =& f(x_{t-1}, x_t, x_{t+1}, d_t, v_t)
\end{align}
$$

where $x_t$ is the time-$t$ vector of _aggregate_ variables, with each aggregate variables $x_{i,t}$ indexed over $i \in 1,2, \dots, N$. $v_t$ is the agents decision functions over the complete state space and $d_t$ is the distribution accross agents.

## The steady state

Let the steady state values be denoted by a bar, i.e. $\bar{x}$, $\bar{d}$ and $\bar{v}$. The steady state is often not unique. In this case it is necessary to fix some variables to given values. Denote the set of indices of the values to be fixed as $K$ and the fixed values as $\hat{x}$. Further, let $b(x)$ be a function that sets $x_K=\hat{x}$, i.e.
$$
b(x) = \{x_{\not\in K} \land \hat{x}\}.
$$

The steady state then requires
$$
\begin{align}
\bar{v} =& V(\bar{x}, \bar{x}, \bar{x}, \bar{v})\\
\bar{d} =& D(\bar{x}, \bar{x}, \bar{x}, \bar{v}, \bar{d})\\
0 =& f(\bar{x}, \bar{x}, \bar{x}, \bar{d}, \bar{v})\\
\bar{x} =& b(\bar{x})
\end{align}
$$

For a given guess on $\bar{x}$, the first equation can be solved for $\bar{v}$ via backwards iteration. Denote the function that solves this as $\bar{v}=\bar{V}(\bar{x})$.
Given $\bar{x}$ and $\bar{v}$, the second equation can be solved via forward iteration (or via eigenvector, but eigenvectors are not yet available for automatic differentiation). Denote this solver as $\bar{d} = \bar{D}(\bar{x},\bar{v})$. Define $\bar{f}$ equivalently.

Combining those, we have a function $H$ such that
$$
H(\bar{x}) = \bar{f}\left(b(\bar{x}), \bar{D}\left(b(\bar{x}), \bar{V}(b(\bar{x}))\right), \bar{V}(b(\bar{x}))\right) = 0
$$

We can calculate the Jacobian of $H$ using automatic differentiation and, starting with some guess $X^j$ on $\bar{x}$, we can iterate on $H$ using a Newton method. During iteration we use the Moore–Penrose inverse for the inversion Jacobian since $\bar{f}$ may not have full rank, in which case it is necessary that $K \neq \{\}$, from which it follows that $b(x)$ certainly also does not have full rank.


## Dynamic solution

Let, for the sake of generality, letters with a tilde denote the complete expected time series of a variable up to some distant point in time $T$, e.g $\tilde{x} = \{x_{t}\}_{t=0}^T$. The above system can be written as

$$
\begin{align}
v_t =& V(\tilde{x}, v_{t+1})\\
d_t =& D(\tilde{x}, \tilde{v}, d_{t-1})\\
0 =& F(\tilde{x}, \tilde{d}, \tilde{v})
\end{align}
$$

with $F=\{f(x_{t-1},x_t,x_{t+1}, d_t, v_t)\}_{t=1}^T$. 

$V$ often take the form of a value function. Note that the above assumes that $V$ is independent of the distribution at any point in time. Assuming that after $T$ we are back in the steady state, the sequence $\tilde{v}$ can be found by starting with
$$
v_T = V(\tilde{x}, \bar{v})
$$
and iterating backwards until we arrive at $v_0$.

Now taking $v_0$ as given and assuming we are also in the steady state in $t=0$, we can start with $d_{-1}=\bar{d}$ and iterate $D$ forward:

$$
d_0 = D(\tilde{x}, \tilde{v}, \bar{d})
$$
until we arrive at $d_T$. This yields $\tilde{d}$.

Denote the function that generates $\tilde{v}$ from $\tilde{x}$ as $\hat{V}$, and a function that generates $\tilde{d}$ from $\tilde{x}$ and $\tilde{v}$ as $\hat{D}$. Then we can define
$$
G(\tilde{x}) = F\left(\tilde{x}, \hat{D}\left(\tilde{x}, \hat{V}(\tilde{x})\right), \hat{V}(\tilde{x})\right) = 0
$$
as a function that only depends on the sequence $\tilde{x}$.

Given that the jacobian $J_G$ of $G(\cdot)$ is available via automatic differentiation, we can then use a Newton method to solve for $\tilde{x}$ directly without having to explicitely keep track of any distribution variable.